# Analise de Adstock e Saturacao

Notebook dedicado a explorar os efeitos de adstock e saturacao nos canais de midia.


In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 1. Visualizando o Efeito Adstock

O adstock modela o carry-over da publicidade. Vamos comparar diferentes taxas de decay.


In [ ]:
from src.features.adstock import geometric_adstock, weibull_adstock

impulse = pd.Series([1000] + [0] * 19, name="impulse")

fig = go.Figure()
fig.add_trace(go.Bar(x=list(range(20)), y=impulse.values, name="Investimento original", opacity=0.3))

for decay in [0.3, 0.5, 0.7, 0.9]:
    adstocked = geometric_adstock(impulse, decay_rate=decay)
    fig.add_trace(go.Scatter(x=list(range(20)), y=adstocked.values, name=f"Decay={decay}", mode="lines+markers"))

fig.update_layout(
    title="Efeito do Adstock Geometrico com Diferentes Taxas de Decay",
    xaxis_title="Semana",
    yaxis_title="Valor",
    template="plotly_white",
)
fig.show()

## 2. Adstock Weibull vs Geometrico


In [ ]:
impulse = pd.Series([1000] + [0] * 19, name="impulse")

geo = geometric_adstock(impulse, decay_rate=0.6)
weib1 = weibull_adstock(impulse, shape=1.5, scale=5.0)
weib2 = weibull_adstock(impulse, shape=3.0, scale=5.0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(20)), y=geo.values, name="Geometrico (decay=0.6)"))
fig.add_trace(go.Scatter(x=list(range(20)), y=weib1.values, name="Weibull (shape=1.5, scale=5)"))
fig.add_trace(go.Scatter(x=list(range(20)), y=weib2.values, name="Weibull (shape=3.0, scale=5)"))

fig.update_layout(
    title="Comparacao: Adstock Geometrico vs Weibull",
    xaxis_title="Semana",
    yaxis_title="Efeito",
    template="plotly_white",
)
fig.show()

## 3. Curvas de Saturacao

As curvas de saturacao modelam retornos decrescentes.


In [ ]:
from src.features.saturation import (
    exponential_saturation,
    hill_saturation,
    logistic_saturation,
    power_saturation,
)

x = np.linspace(0, 1, 200)

fig = make_subplots(rows=2, cols=2, subplot_titles=(
    "Exponencial", "Hill", "Logistica", "Potencia"
))

for lambd in [1, 3, 5, 10]:
    fig.add_trace(go.Scatter(x=x, y=exponential_saturation(x, lambd), name=f"lambda={lambd}"), row=1, col=1)

for gamma in [0.2, 0.4, 0.6, 0.8]:
    fig.add_trace(go.Scatter(x=x, y=hill_saturation(x, alpha=2.0, gamma=gamma), name=f"gamma={gamma}"), row=1, col=2)

for steep in [3, 5, 10, 20]:
    fig.add_trace(go.Scatter(x=x, y=logistic_saturation(x, midpoint=0.5, steepness=steep), name=f"steep={steep}"), row=2, col=1)

for exp in [0.3, 0.5, 0.7, 0.9]:
    fig.add_trace(go.Scatter(x=x, y=power_saturation(x, exponent=exp), name=f"exp={exp}"), row=2, col=2)

fig.update_layout(height=700, title_text="Curvas de Saturacao", template="plotly_white")
fig.show()

## 4. Otimizacao de Decay por Canal


In [ ]:
from src.ingestion.loader import DataLoader
from src.features.adstock import AdstockTransformer
from src.config import MEDIA_CHANNELS

loader = DataLoader()
df = loader.generate_sample_data(n_weeks=104)

transformer = AdstockTransformer(media_channels=MEDIA_CHANNELS)
optimal_decays = transformer.optimize_all_channels(df, target_column="revenue")

print("Decays otimos por canal:")
for ch, decay in optimal_decays.items():
    print(f"  {ch}: {decay:.2f}")

## 5. Nivel de Saturacao Atual


In [ ]:
from src.features.saturation import SaturationTransformer

sat_transformer = SaturationTransformer(media_channels=MEDIA_CHANNELS)
df_sat = sat_transformer.transform(df)
levels = sat_transformer.get_saturation_levels(df_sat)

print("Nivel de saturacao atual por canal:")
for ch, level in sorted(levels.items(), key=lambda x: x[1], reverse=True):
    bar = "#" * int(level * 50)
    print(f"  {ch:20s}: {level:.3f} |{bar}|")

---

*"A natureza nao faz saltos." - Leibniz*
